In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


%matplotlib inline
plt.style.use('default')

# Очистка данных. Закрепление знаний (юнит 8)

Вам предоставлен [набор данных](https://lms.skillfactory.ru/assets/courseware/v1/6559ab1e1d17acac79bec5dc8052261b/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/diabetes_data.zip), первоначально полученный в Национальном институте диабета, болезней органов пищеварения и почек. 

Этот набор данных создан для того, чтобы на основе определённых диагностических измерений предсказать, есть ли у пациента диабет. 

На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года индейского происхождения Пима.

Прочитаем наши данные и выведем первые пять строк таблицы:

In [4]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


## Описание данных

* `Pregnancies` — количество беременностей.<br><br>
* `Glucose` — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.<br><br>
* `BloodPressure` — диастолическое артериальное давление (мм рт. ст.).<br><br>
* `SkinThickness` — толщина кожной складки трицепса (мм).<br><br>
* `Insulin` — двухчасовой сывороточный инсулин (ме Ед/мл).<br><br>
* `BMI` — индекс массы тела ($\frac{вес\ в\ кг}{(рост\ в\ м)^2}$).<br><br>
* `DiabetesPedigreeFunction` — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).<br><br>
* `Age` — возраст.<br><br>
* `Outcome` — наличие диабета (0 — нет, 1 — да).<br><br>

Предварительно вы можете провести небольшой разведывательный анализ: посмотреть на распределения признаков и оценить их взаимосвязь с признаком наличия диабета.

## 8.1

Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [5]:
df = diabetes.copy()
df = df.drop_duplicates()
df.shape[0]

768

## 8.2

Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

Примечание: дальнейшая работа производится с очищенными от дубликатов и неинформативных признаков данных.

In [6]:
linfcols = [
    col for col in df.columns if (
        df[col].value_counts(normalize=True).max() > 0.95 or
        df[col].nunique() / df[col].count() > 0.95)
]

df = df.drop(linfcols, axis=1)
print(linfcols)

['Gender']


## 8.3

Попробуйте найти пропуски в данных с помощью метода `isnull()`.

Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin` и `BMI` обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!

Замените все записи, равные 0, в столбцах `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin` и `BMI` на символ пропуска. Его вы можете взять из библиотеки `numpy`: `np.nan`.

Какая доля пропусков содержится в столбце `Insulin`? Ответ округлите до сотых.

__Примечание к решению:__ Для проверки альтернативного варианта решения 
необходимо перезапустить ноутбук

In [7]:
# Проверка доли значений равных 0 в столбце
print("Доля значений равных 0:", 
      round(df[df['Insulin'] == 0].shape[0] / df.shape[0], 2))

Доля значений равных 0: 0.49


### Вариант 1 (replace + dict comprehension)

In [8]:
cols_for_replace = [
    'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI'
]

df = df.replace({col: {0: np.nan} for col in cols_for_replace})

print("Доля значений NaN:",
    round(df['Insulin'].isna().sum() / df.shape[0], 2))

Доля значений NaN: 0.49


### Вариант 2 (apply)

In [9]:
cols_for_replace = [
    'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI'
]

for col in cols_for_replace:
    df[col] = df[col].apply(
        lambda x: np.nan if x == 0 else x
    ) 

print("Доля значений NaN:",
    round(df['Insulin'].isna().sum() / df.shape[0], 2))

Доля значений NaN: 0.49


### Вариант 3 (фильтрация)

In [10]:
cols_for_replace = [
    'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI'
]

for col in cols_for_replace:
    df.loc[df[col] == 0, col] = np.nan
    
print("Доля значений NaN:",
    round(df['Insulin'].isna().sum() / df.shape[0], 2))

Доля значений NaN: 0.49


## 8.4

Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [11]:
cols_for_drop = [
    col for col in df.columns 
        if df[col].isna().sum() / df.shape[0] >= 0.3
]

df = df.drop(columns=cols_for_drop)
df.shape[1]

8

## 8.5

Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?



In [12]:
df = df.dropna(thresh=6)
df.shape[0]

761

## 8.6

В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце `SkinThickness`? Ответ округлите до десятых.

In [13]:
values = {col: df[col].median() for col in df.columns}
df = df.fillna(values)
df['SkinThickness'].mean().round(1)

29.1

## 8.7

Сколько выбросов найдёт классический метод межквартильного размаха в признаке `SkinThickness`?

In [14]:
def outliers_iqr_mod(
        data: pd.DataFrame, 
        feature: str,
        log_scale: bool=False, 
        left: float=1.5, 
        right: float=1.5
) -> tuple:
    
    y = 1 if data[feature].min() == 0 else 0
        
    x = np.log(data[feature]+y) if log_scale else data[feature]
    
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    
    return outliers, cleaned

print(outliers_iqr_mod(df, 'SkinThickness')[0].shape[0])

87


## 8.8

Сколько выбросов найдёт классический метод z-отклонения в признаке `SkinThickness`?

In [15]:
def outliers_z_score_mod(
        data: pd.DataFrame, 
        feature: str, 
        log_scale: bool=False,
        left: int=3,
        right: int=3
) -> tuple:
    
    y = 1 if data[feature].min() == 0 else 0
    x = np.log(data[feature]+y) if log_scale else data[feature]
        
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left*sigma
    upper_bound = mu + right*sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    
    return outliers, cleaned

print(outliers_z_score_mod(df, 'SkinThickness')[0].shape[0])

4


## 8.9

На приведённой гистограмме показано распределение признака `DiabetesPedigreeFunction`. Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. Найдите сначала число выбросов в признаке `DiabetesPedigreeFunction` с помощью классического метода межквартильного размаха.

Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). Какова разница между двумя этими числами (вычтите из первого второе)?

![](pictures/dst-3-unit-1-mod-14-35.png)

In [16]:
n_outliers = outliers_iqr_mod(
    df, 'DiabetesPedigreeFunction'
)[0].shape[0]

n_outliers_log = outliers_iqr_mod(
    df, 'DiabetesPedigreeFunction',
    log_scale=True
)[0].shape[0]

print(n_outliers - n_outliers_log)

29


## Задание на самопроверку

Имеются [две базы](https://lms.skillfactory.ru/assets/courseware/v1/958d35ff25f2486f65613da4459e6647/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/Data_TSUM.xlsx) данных (два листа Excel-файла): база с ценами конкурентов (`Data_Parsing`) и внутренняя база компании (`Data_Company`).

В базе парсинга есть два `id`, однозначно определяющие товар: `producer_id` и `producer_color`.

В базе компании есть два аналогичных поля: `item_id` и `color_id`.

Нам известно, что коды в двух базах отличаются наличием набора служебных символов. В базе парсинга встречаются следующие символы: `_`, `-`, `~`, `\`, `/`.

Необходимо:

1. Считать данные из `Excel` в `DataFrame` (`Data_Parsing`) и (`Data_Company`).<br><br>
1. Подтянуть к базе парсинга данные из базы компании (`item_id`, `color_id`, `current_price`) и сформировать столбец разницы цен в % (цена конкурента к нашей цене).<br><br>
1. Определить сильные отклонения от среднего в разности цен в пределах бренда-категории (то есть убрать случайные выбросы, сильно искажающие сравнение). Критерий — по вкусу, написать комментарий в коде.<br><br>
1. Записать новый файл `Excel` с базой парсинга, приклееными к ней столбцами из пункта 2 и с учётом пункта 3 (можно добавить столбец `outlier` и проставить `Yes` для выбросов).

In [17]:
df_dict = pd.read_excel(
    'data/Data_TSUM.xlsx',
    sheet_name=['Data_Parsing', 'Data_Company']
    )
data_parsing = df_dict['Data_Parsing']
data_company = df_dict['Data_Company']

In [18]:
data_parsing['producer_id'] = data_parsing['producer_id'].apply(
    lambda val: ''.join(s for s in val if s.isalnum())
)

data_parsing['producer_color'] = data_parsing['producer_color'].apply(
    lambda val: ''.join(s for s in val if s.isalnum())
)


data_parsing['id'] = data_parsing.agg(
    lambda x: f"{x['producer_id']}_{x['producer_color']}", axis=1
)

data_company['id'] = data_company.agg(
    lambda x: f"{x['item_id']}_{x['color_id']}", axis=1
)

data_parsing = data_parsing.merge(data_company)

data_parsing['price_diff'] = data_parsing['current price'] / data_parsing['price']

In [19]:
data_parsing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 0 to 74
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   brand           75 non-null     object 
 1   Category        75 non-null     object 
 2   producer_id     75 non-null     object 
 3   producer_color  75 non-null     object 
 4   price           75 non-null     int64  
 5   id              75 non-null     object 
 6   item_id         75 non-null     object 
 7   color_id        75 non-null     object 
 8   current price   75 non-null     int64  
 9   price_diff      75 non-null     float64
dtypes: float64(1), int64(2), object(7)
memory usage: 6.4+ KB


In [21]:
fig = px.box(
    data_parsing, 
    x="price_diff", 
    y="brand",
    title = 'Разница цен компании и конкурентов'
)
fig.update_layout(
    xaxis=dict(
        tickformat='.0%',
        title_text='Цены конкурентов от цен компании, %'    
    ),
    yaxis_title_text='Бренд',
    title_x=0.5,
    width=800,
    height=600
)
fig.show()

In [30]:
fig = px.histogram(
    data_frame=data_parsing[data_parsing['brand']=='Stone Island'],
    title='Распределение',
    labels={
        'price_Diff': 'Разница в цене, %', 
        'brand': 'Бренд',
    },
    # category_orders={'Outflow': ['Loyal', 'Departed']},
    x='price_diff',
    marginal='box',
    color='brand',
    # nbins=30,
    width=1000,
    height=600,
    template='plotly'
)

fig.update_layout(
    title_x=0.5,
    yaxis_title_text='Количество товаров, шт.'
)
fig.show()